In [ ]:
from tensorflow import keras
import pydicom
import numpy as np

In [ ]:
model = keras.models.load_model('path/to/location')

In [ ]:
def full_cta_model_evaluation(image, model, z_stride, which_prediction):
    '''
    Helper function, which given the original ct image of a plaque will give back the masks for the DL segmentation.
    '''
    # Shape of original images
    size_X = image.shape[2]
    size_Y = image.shape[1]
    size_Z = image.shape[0]

    image_paded = np.zeros((size_Z + 24,
                            size_Y,
                            size_X))

    image_paded[:size_Z, :size_Y, :size_X] = image / 512

    prediction_array = np.zeros((size_Z + 24,
                                 size_Y,
                                 size_X))

    coverage_array = np.zeros((size_Z + 24,
                               size_Y,
                               size_X))

    # Containers for batch predictions
    patch_boundaries_list = []
    counter = 0

    # Stride along Z axis:  
    for z_start in range(0, prediction_array.shape[2], z_stride):
        z_end = z_start + 24
        if (np.count_nonzero(image[z_start:z_end, :, :]) > 1):
            patch_boundaries_list.append([z_start, z_end])
    for patch_index in range(0, len(patch_boundaries_list)):
        # patch_boundaries in current batch
        temporal_boundaries = patch_boundaries_list[patch_index]
        temp_patches = []
        # Extracting patches for prediction
        current_patch = image_paded[temporal_boundaries[0]:temporal_boundaries[1],
                        16:48,
                        16:48]
        current_patch = np.expand_dims(current_patch, axis=0)
        # Updating prediction_array and coverage_array
        predicted_patch = model.predict(np.expand_dims(current_patch, axis=-1))

        # 0 belső maszk 1 külső maszk 2 differencia

        prediction = predicted_patch[which_prediction]

        prediction = np.reshape(prediction, [24, 32, 32])

        prediction_array[temporal_boundaries[0]:temporal_boundaries[1],
        16:48,
        16:48] += prediction

        # print(prediction_array[32, 32, 32])

        coverage_array[temporal_boundaries[0]:temporal_boundaries[1],
        :,
        :] += 1

    coverage_array = np.maximum(coverage_array, np.ones(coverage_array.shape))
    # Taking the average prediction value for the pixels
    prediction_array = np.divide(prediction_array, coverage_array)
    # print(prediction_array[32,32,32])
    # Removing the prediction values outside of the CT  
    prediction_array = prediction_array[0:size_Z, 0:size_Y, 0:size_X]

    # The average prediction value is continuous between 0 and 1,   
    # so for the segmentation we have to threshold it   
    prediction_array = (prediction_array > 1 / 2) * 1

    return prediction_array

In [ ]:
def mask_from_dicom(contour_file_name):
    ds = pydicom.dcmread(contour_file_name)
    pixels = np.array(ds.pixel_array)
    pixels = np.where(pixels > 0, 1, 0)
    return pixels

In [ ]:
def one_vessel(image_path):
    '''
    retrun DL segmentation and original segmentation of one plaque.
    '''
    image_dicom=pydicom.dcmread(image_path)
    image = image_dicom.pixel_array

    dif_prediction = full_cta_model_evaluation(image = image, model = model, z_stride = 12, which_prediction=2) 
    #contour of plaque area, segmented by DL
    mask_1_prediction= full_cta_model_evaluation(image = image, model = model, z_stride = 12, which_prediction=0)
    #contour of inner vessel wall segmented by DL
    mask_2_prediction= full_cta_model_evaluation(image = image, model = model, z_stride = 12, which_prediction=1)  
    #contour of outer vessel wall segmented by DL

    mask_1_path = os.path.join(basic_path, image_name[:-13]+"Contour1.dcm") 
    #if the naming process of the mevislab code is not ovewritten, this loads the relevant files
    mask_2_path = os.path.join(basic_path, image_name[:-13]+"Contour2.dcm")
    mask_1 = mask_from_dicom(mask_1_path) #contour of inner vessel wall of the original segmentation
    mask_2 = mask_from_dicom(mask_2_path) #contour of outer vessel wall of the original segmentation
    dif_test=mask_2-mask_1 #contour of the plaque area of the original segmentation
        
    return image, dif_test, dif_prediction, mask_1_prediction, mask_2_prediction, mask_1, mask_2

In [ ]:
image, dif_test, dif_prediction, mask_1_prediction, mask_2_prediction, mask_1, mask_2 = one_vessel(image_path)